In [18]:
import pandas as pd
import numpy as np
import pickle

In [21]:
def load_data(day, time):
    # load data
    print(f"Loading accident and congestion data for {day}, {time}")
    accident_df = pd.read_csv(
        f"processed_data/accident_tables/{day}_{time}_accidents.csv"
    )
    congestion_df = pd.read_csv(
        f"processed_data/congestion_tables/{day}_{time}_congestion.csv"
    )

    # process data
    print(f"Processing data")
    accident_df["date"] = pd.to_datetime(accident_df["date"])
    congestion_df["date"] = pd.to_datetime(congestion_df["date"])

    test_filter = accident_df["date"].dt.year == 2023

    A_indices = accident_df.index[test_filter].tolist()

    accident_df_filtered = accident_df[test_filter].drop(
        "date", axis=1
    )
    congestion_df_filtered = congestion_df[test_filter].drop(
        "date", axis=1
    )

    to_drop = (
        accident_df_filtered.isna().any(axis=1)
        | congestion_df_filtered.isna().any(axis=1)
        | (accident_df_filtered == 0).all(axis=1)
    )

    accident_df_filtered = accident_df_filtered[~to_drop]
    congestion_df_filtered = congestion_df_filtered[~to_drop]

    # process accidents
    accident_np = accident_df_filtered.to_numpy()
    accident_counts = accident_np.sum(axis=1)
    P = accident_np / accident_counts[:, np.newaxis]

    return P, A_indices

In [22]:
P_test, A_indices = load_data("wd", "night")

Loading accident and congestion data for wd, night
Processing data


In [36]:
P_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(120, 300))

In [23]:
with open('results/model_1_raw.pickle', 'rb') as f:
    results = pickle.load(f)

In [29]:
result = results['wd']['early'][20]

X_opt = result['X']
y_opt = result['y']
A_traffics = np.load(f"processed_data/accident_tables/A_traffics_wd_early.npy")

In [ ]:
def evaluate_on_test(A_test, P_test, X_opt, y_opt):
    scenario_times = []
    for i in range(len(A_test)):
        # Same objective expression as training
        T_i = ((A_test[i] * X_opt) @ P_test[i, :]).sum()
        scenario_times.append(T_i)
        
    return np.mean(scenario_times)


In [35]:
P_test.shape

(120, 300)

In [30]:
evaluate_on_test(A_traffics[A_indices[0]], P_test, X_opt, y_opt)

IndexError: index 120 is out of bounds for axis 0 with size 120